In [1]:
print ("hi")

hi


In [ ]:
print ("bye")

bye


In [ ]:
print ("nooo")

In [ ]:
import xarray as xr
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

# =====================================================
# USER SETTINGS
# =====================================================

data_2025 = "/home/nilay/SSD Data Downloads/ERA5 October Hourly Unziped"

clim_tp_dir = "/media/nilay/HDD/IIT KGP 2025/PHD/ERA5_Downloads/ERA5_1980-2017_CLIM_Unzipped/tp"
clim_wind_dir = "/media/nilay/HDD/IIT KGP 2025/PHD/ERA5_Downloads/ERA5_1980-2017_CLIM_Unzipped/vimd"

india_shp = "/media/nilay/HDD/IIT KGP 2025/PHD/Shapefiles/India Shapefile With Kashmir/India Shape/india_st.shp"
wd_file = "/home/nilay/SSD Data Downloads/wd_v5-era5-1950-2025.csv"

lat_min, lat_max = 5, 40
lon_min, lon_max = 60, 100

# ================= SWITCHES =================

PLOT_PRECIP = True
PLOT_WIND = True
PLOT_WD = True

WIND_LEVEL = 850
WIND_SKIP = 10

# =====================================================
# LOAD 2025 DATA
# =====================================================

files_2025 = sorted(glob.glob(f"{data_2025}/*.nc"))

ds = xr.open_mfdataset(files_2025, combine="by_coords")

if ds.longitude.max() > 180:
    ds = ds.assign_coords(longitude=((ds.longitude + 180) % 360) - 180)
    ds = ds.sortby("longitude")

ds = ds.sel(
    latitude=slice(lat_max, lat_min),
    longitude=slice(lon_min, lon_max)
)

# =====================================================
# PRECIP ANOMALY
# =====================================================

if PLOT_PRECIP:

    rain = ds["tp"] * 1000
    daily_rain = rain.resample(valid_time="1D").sum()

    # ---- climatology precipitation ----
    clim_files = sorted(glob.glob(f"{clim_tp_dir}/*.nc"))
    clim = xr.open_mfdataset(clim_files, combine="by_coords")

    clim_rain = (clim["tp"] * 1000).groupby("time.day").mean("time")

    rain_anom = daily_rain.groupby("valid_time.day") - clim_rain

# =====================================================
# WIND ANOMALY
# =====================================================

if PLOT_WIND:

    u = ds["u"].sel(pressure_level=WIND_LEVEL, method="nearest")
    v = ds["v"].sel(pressure_level=WIND_LEVEL, method="nearest")

    daily_u = u.resample(valid_time="1D").mean()
    daily_v = v.resample(valid_time="1D").mean()

    # ---- climatology wind ----
    clim_files = sorted(glob.glob(f"{clim_wind_dir}/*.nc"))
    clim_wind = xr.open_mfdataset(clim_files, combine="by_coords")

    clim_u = clim_wind["u"].sel(
        pressure_level=WIND_LEVEL,
        method="nearest"
    ).groupby("time.day").mean("time")

    clim_v = clim_wind["v"].sel(
        pressure_level=WIND_LEVEL,
        method="nearest"
    ).groupby("time.day").mean("time")

    u_anom = daily_u.groupby("valid_time.day") - clim_u
    v_anom = daily_v.groupby("valid_time.day") - clim_v

    print("Using wind level:", float(u.pressure_level.values), "hPa")

# =====================================================
# WD DATA
# =====================================================

if PLOT_WD:

    wd_df = pd.read_csv(wd_file)

    wd_df["datetime"] = pd.to_datetime(
        wd_df[["year", "month", "day", "hour"]]
    )

    wd_df["date"] = wd_df["datetime"].dt.date

# =====================================================
# COLORMAP
# =====================================================

rain_levels = np.arange(-100, 110, 10)
rain_cmap = plt.cm.RdBu_r
rain_norm = mcolors.BoundaryNorm(rain_levels, rain_cmap.N)

# =====================================================
# PLOTTING
# =====================================================

fig = plt.figure(figsize=(18, 20))
proj = ccrs.PlateCarree()

shape_feature = cfeature.ShapelyFeature(
    shpreader.Reader(india_shp).geometries(),
    proj,
    edgecolor="black",
    facecolor="none",
    linewidth=0.6
)

time_coord = rain_anom.valid_time

for i, day in enumerate(time_coord):

    ax = plt.subplot(6, 6, i+1, projection=proj)
    current_date = pd.to_datetime(str(day.values)).date()

    # ================= PRECIP =================

    if PLOT_PRECIP:

        rain_plot = ax.pcolormesh(
            rain_anom.longitude,
            rain_anom.latitude,
            rain_anom.sel(valid_time=day),
            cmap=rain_cmap,
            norm=rain_norm,
            shading="auto"
        )

    # ================= WIND =================

    if PLOT_WIND:

        u_day = u_anom.sel(valid_time=day)
        v_day = v_anom.sel(valid_time=day)

        ax.quiver(
            u_day.longitude.values[::WIND_SKIP],
            u_day.latitude.values[::WIND_SKIP],
            u_day.values[::WIND_SKIP, ::WIND_SKIP],
            v_day.values[::WIND_SKIP, ::WIND_SKIP],
            scale=400,
            width=0.003,
            transform=ccrs.PlateCarree()
        )

    # ================= MAP =================

    ax.add_feature(cfeature.COASTLINE.with_scale("110m"), linewidth=0.5)
    ax.add_feature(shape_feature)

    # ================= WD =================

    if PLOT_WD:

        wd_today = wd_df[wd_df["date"] == current_date]

        if not wd_today.empty:

            for track_id, track in wd_today.groupby("track_id"):

                track_box = track[
                    (track["lon"] >= lon_min) &
                    (track["lon"] <= lon_max) &
                    (track["lat"] >= lat_min) &
                    (track["lat"] <= lat_max)
                ]

                if len(track_box) > 0:

                    ax.plot(
                        track_box["lon"],
                        track_box["lat"],
                        color="black",
                        linewidth=2,
                        transform=ccrs.PlateCarree()
                    )

    ax.set_title(str(day.values)[:10], fontsize=8)

# =====================================================
# COLORBAR
# =====================================================

cax = fig.add_axes([0.27, 0.12, 0.5, 0.02])

cbar = fig.colorbar(
    rain_plot,
    cax=cax,
    orientation="horizontal"
)

cbar.set_label("Rainfall Anomaly (mm)", fontsize=12)

plt.show()